In [4]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import pandas as pd
%matplotlib inline

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
address = 'data/Data_SOLMAT.xlsx'
x = []
y = []
df = pd.read_excel(address,sheet_name = 'Normalized_table',usecols = [0],names = None)
df_li_Time = df.values.tolist()
df = pd.read_excel(address,sheet_name = 'Normalized_table',usecols = [1],names = None)
df_li_L = df.values.tolist()

In [13]:
#Set dependent variable
for s_li in df_li_Time:
    x.append(s_li[0])
x = np.array([x]).T

#Set indepdent varibles
for a_li in df_li_L:
    y.append(a_li[0])
    
y = np.array([y]).T
print(x,y)

[[0.2]
 [0.6]
 [1. ]
 [0. ]
 [0.4]
 [0.8]
 [0.6]
 [1. ]
 [0.4]
 [0.8]] [[0.18181818]
 [0.09090909]
 [0.63636364]
 [0.45454545]
 [0.36363636]
 [0.27272727]
 [0.27272727]
 [0.        ]
 [0.63636364]
 [0.09090909]]
